# 9-9. 프로젝트 - Movielens 영화 추천 실습

이전 스텝에서 배운 MF 모델 학습 방법을 토대로, 내가 좋아할 만한 영화 추천 시스템을 제작해 보겠습니다.

cloud jupyter 를 열고 aiffel/recommendata_iu/ 디렉토리에 python3로 ipynb파일을 생성 해 주세요 jupyter notebook 환경에서 프로젝트 코드를 작성하고 제출하시면 됩니다.

이번에 활용할 데이터셋은 추천시스템의 MNIST라고 부를만한 Movielens 데이터입니다.

- 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.
- 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해볼 수 있습니다.
- 별점을 **시청횟수** 로 해석해서 생각하겠습니다.
- 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.

Cloud Storage에 미리 업로드 된 ml-1m폴더 내 파일을 심볼릭 링크로 개인 storage에 연결 해 줍니다.

Cloud shell에서 아래 명령어를 입력해주세요.

- mkdir -p ~/aiffel/recommendata_iu/data/ml-1m
- ln -s ~/data/ml-1m/* ~/aiffel/recommendata_iu/data/ml-1m

목차
---
1) 데이터 준비와 전처리.  
2) 분석해 봅시다.  
3) 내가 선호하는 영화를 5가지 골라서 ratings에 추가해 줍시다.  
4) CSR matrix를 직접 만들어 봅시다.  
5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.  
6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.  
7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.  
8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다

## 1) 데이터 준비와 전처리.
---
Movielens 데이터는 rating.dat 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있습니다.

In [27]:
# 필요한 모듈 import
import numpy as np
import pandas as pd
import os

In [28]:
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [29]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


3점 이상인 영화의 비율이 83.63% 임을 알 수 있습니다.

In [30]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [31]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [32]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


여기까지가 전처리입니다. 이후에는 이전 스텝에 소개했던 것과 동일한 방식으로 MF model을 구성하여 내가 좋아할 만한 영화를 추천해 볼 수 있습니다.

In [33]:
# 영화 제목 검색을 쉽게하기 위해 아티스트 문자열을 소문자로 바꿔 줍시다.
movies['title'] = movies['title'].str.lower() # 검색을 쉽게하기 위해 아티스트 문자열을 소문자로 바꿔줍시다.
movies.head(10)

,movie_id,title,genre
0,1,toy story (1995),Animation|Children's|Comedy
1,2,jumanji (1995),Adventure|Children's|Fantasy
2,3,grumpier old men (1995),Comedy|Romance
3,4,waiting to exhale (1995),Comedy|Drama
4,5,father of the bride part ii (1995),Comedy
5,6,heat (1995),Action|Crime|Thriller
6,7,sabrina (1995),Comedy|Romance
7,8,tom and huck (1995),Adventure|Children's
8,9,sudden death (1995),Action
9,10,goldeneye (1995),Action|Adventure|Thriller


In [34]:
# ratings 와 movies 데이터 프레임 합치기
data = ratings.join(movies.set_index('movie_id'), on='movie_id')

data.head()

,user_id,movie_id,counts,timestamp,title,genre
0,1,1193,5,978300760,one flew over the cuckoo's nest (1975),Drama
1,1,661,3,978302109,james and the giant peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,my fair lady (1964),Musical|Romance
3,1,3408,4,978300275,erin brockovich (2000),Drama
4,1,2355,5,978824291,"bug's life, a (1998)",Animation|Children's|Comedy


In [35]:
# 필요없는 timestamp 을 지워줍니다.
del data['timestamp']
data.head()

,user_id,movie_id,counts,title,genre
0,1,1193,5,one flew over the cuckoo's nest (1975),Drama
1,1,661,3,james and the giant peach (1996),Animation|Children's|Musical
2,1,914,3,my fair lady (1964),Musical|Romance
3,1,3408,4,erin brockovich (2000),Drama
4,1,2355,5,"bug's life, a (1998)",Animation|Children's|Comedy


In [36]:
# user_id가 1인 사용자가 어떤 영화를 시청했는지 확인해봅니다.
movie_user_id_1 = data[data['user_id']==1]['title']
movie_user_id_1

0                one flew over the cuckoo's nest (1975)
1                      james and the giant peach (1996)
2                                   my fair lady (1964)
3                                erin brockovich (2000)
4                                  bug's life, a (1998)
5                            princess bride, the (1987)
6                                        ben-hur (1959)
7                             christmas story, a (1983)
8                snow white and the seven dwarfs (1937)
9                              wizard of oz, the (1939)
10                          beauty and the beast (1991)
11                                          gigi (1958)
12                        miracle on 34th street (1947)
13                      ferris bueller's day off (1986)
14                           sound of music, the (1965)
15                                     airplane! (1980)
16                                        tarzan (1999)
17                                         bambi

## 2) 분석해 봅시다.
---
- ratings에 있는 유니크한 영화 개수
- ratings에 있는 유니크한 사용자 수
- 가장 인기 있는 영화 30개(인기순)

### 2-1. ratings에 있는 유니크한 영화 개수

In [37]:
ratings['movie_id'].nunique()

3628

### 2-2. ratings에 있는 유니크한 사용자 수

In [38]:
ratings['user_id'].nunique()

6039

### 2-3. 가장 인기 있는 영화 30개(인기순)

In [39]:
data_count = data.groupby('title')['user_id'].count()
data_count.sort_values(ascending=False).head(30)

title
american beauty (1999)                                   3211
star wars: episode iv - a new hope (1977)                2910
star wars: episode v - the empire strikes back (1980)    2885
star wars: episode vi - return of the jedi (1983)        2716
saving private ryan (1998)                               2561
terminator 2: judgment day (1991)                        2509
silence of the lambs, the (1991)                         2498
raiders of the lost ark (1981)                           2473
back to the future (1985)                                2460
matrix, the (1999)                                       2434
jurassic park (1993)                                     2413
sixth sense, the (1999)                                  2385
fargo (1996)                                             2371
braveheart (1995)                                        2314
men in black (1997)                                      2297
schindler's list (1993)                                  2257
pr

In [40]:
# 유저별 몇 편의 영화를 시청했는지에 대한 통계
user_count = data.groupby('user_id')['movie_id'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: movie_id, dtype: float64

## 3) 내가 선호하는 영화를 5가지 골라서 ratings에 추가해 줍시다.
---

우선 keyword로 영화를 찾고 movie_id를 이용해서 영화를 찾습니다.

In [41]:
# keyword 로 영화 찾기
movies[movies['title'].str.lower().str.contains('terminator', regex=False)]

,movie_id,title,genre
585,589,terminator 2: judgment day (1991),Action|Sci-Fi|Thriller
1220,1240,"terminator, the (1984)",Action|Sci-Fi|Thriller


In [42]:
# keyword 로 영화 찾기
movies[movies['title'].str.lower().str.contains('shawshank', regex=False)]

,movie_id,title,genre
315,318,"shawshank redemption, the (1994)",Drama


In [43]:
# keyword 로 영화 찾기
movies[movies['title'].str.lower().str.contains('matrix', regex=False)]

,movie_id,title,genre
2502,2571,"matrix, the (1999)",Action|Sci-Fi|Thriller


In [44]:
# keyword 로 영화 찾기
movies[movies['title'].str.lower().str.contains('forrest', regex=False)]

,movie_id,title,genre
352,356,forrest gump (1994),Comedy|Romance|War


In [45]:
# keyword 로 영화 찾기
movies[movies['title'].str.lower().str.contains('toy', regex=False)]

,movie_id,title,genre
0,1,toy story (1995),Animation|Children's|Comedy
1948,2017,babes in toyland (1961),Children's|Fantasy|Musical
2184,2253,toys (1992),Action|Comedy|Fantasy
2411,2480,dry cleaning (nettoyage à sec) (1997),Drama
3017,3086,march of the wooden soldiers (a.k.a. laurel & ...,Comedy
3045,3114,toy story 2 (1999),Animation|Children's|Comedy


In [46]:
# 내가 선호하는 영화 5개 선정
# (한글)영화 이름 /(영어) 영화 이름 / movie_id
# 쇼생크 탈출 / shawshank redemption, the (1994) / 318
# 매트릭스 / matrix, the (1999) / 2571
# 포레스트 검프 / forrest gump (1994) / 356
# 터미네이터 2 / terminator 2: judgment day (1991) / 589
# 토이스토리 / toy story (1995) / 1

# 내가 선호하는 영화 id 리스트화
my_favoite_movie_id = [318, 2571, 356 , 589, 1]

# id 리스트를 title 리스트로 변환
my_favorite_movie = []
for mid in my_favoite_movie_id:
    my_favorite_movie.append(list(movies[movies['movie_id']==mid]['title'])[0])

# 'hojin'이라는 user_id가 위 영화를 5회씩 시청했다고 가정하겠습니다.
my_watchlist = pd.DataFrame({'user_id': ['hojin']*5, 'movie_id': my_favoite_movie_id, 'title': my_favorite_movie, 'counts':[5]*5})

if not data.isin({'user_id':['hojin']})['user_id'].any():  # user_id에 'hojin'이라는 데이터가 없다면
    data = data.append(my_watchlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

data = data[['user_id', 'movie_id', 'title', 'counts']]
data.tail()

,user_id,movie_id,title,counts
0,hojin,318,"shawshank redemption, the (1994)",5
1,hojin,2571,"matrix, the (1999)",5
2,hojin,356,forrest gump (1994),5
3,hojin,589,terminator 2: judgment day (1991),5
4,hojin,1,toy story (1995),5


## 4) CSR matrix를 직접 만들어 봅시다.
---

In [51]:
from scipy.sparse import csr_matrix

In [47]:
# 고유한 유저, 영화를 찾기
user_unique = data['user_id'].unique()
movie_unique = data['title'].unique()

In [48]:
# 유저, 영화 indexing
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [49]:
# indexing 여부 확인
print(user_to_idx['hojin']) 
print(movie_to_idx['toy story (1995)'])

6039
40


In [53]:
# 데이터 컬럼 내 값을 indexing된 값으로 교체

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['new_user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')
    
# movie_to_idx을 통해 title 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = data['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(data):
    print('title column indexing OK!!')
    data['new_movie_id'] = temp_movie_data
else:
    print('movie_id column indexing Fail!!')

data

user_id column indexing OK!!
movie column indexing OK!!


,user_id,movie_id,title,counts,user_id2,movie_id2
0,1,1193,one flew over the cuckoo's nest (1975),5,0,0
1,1,661,james and the giant peach (1996),3,0,1
2,1,914,my fair lady (1964),3,0,2
3,1,3408,erin brockovich (2000),4,0,3
4,1,2355,"bug's life, a (1998)",5,0,4
...,...,...,...,...,...,...
0,hojin,318,"shawshank redemption, the (1994)",5,6039,157
1,hojin,2571,"matrix, the (1999)",5,6039,124
2,hojin,356,forrest gump (1994),5,6039,160
3,hojin,589,terminator 2: judgment day (1991),5,6039,92


In [54]:
# CSR Matrix 생성
num_user = data['user_id2'].nunique()
num_movie = data['movie_id2'].nunique()

# csr_matrix((data, (row_ind, col_ind)), [shape=(M, N)])
csr_data = csr_matrix((data.counts, (data.user_id2, data.movie_id2)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.
---

Matrix Factorization 모델을 implicit 패키지를 사용하여 학습시킵니다.

- implicit 패키지는 암묵적(implicit) dataset을 사용하는 다양한 모델을 굉짱히 빠르게 학습할 수 있는 패키지입니다.
- 이 패키지에서 구현된 als(AlternatingLeastSquares) 모델을 사용합니다.

In [55]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

AlternatingLeastSquares 클래스의 __init__ 파라미터를 살펴보면,

1. factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지
2. regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지
3. use_gpu : GPU를 사용할 것인지
4. iterations : epochs와 같은 의미입니다. 데이터를 몇 번 반복해서 학습할 것인지

참고로 1,4를 늘릴수록 학습을 잘 하게 되지만 과적합의 우려가 있습니다.

In [56]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [57]:
# als 모델은 input으로 item X user 꼴의 matrix를 받기 때문에 Transpose 해줍니다.
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [58]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

## 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.
---

### 6-1) hojin user 벡터와 toy story (1995) 벡터 내적을 통한 선호도 파악

In [65]:
hojin, toy_story = user_to_idx['hojin'], movie_to_idx['toy story (1995)']
hojin_vector, toy_story_vector = als_model.user_factors[hojin], als_model.item_factors[toy_story]

In [66]:
# hojin 벡터
print(hojin_vector.shape)
hojin_vector

(100,)


array([-0.31227317, -0.59012204,  0.8572613 , -0.03003868, -0.01154263,
       -0.37485448,  0.17735931,  0.5475062 ,  0.40671805,  1.5321466 ,
        0.8196254 ,  0.51811105,  0.5252771 , -0.38434356,  0.40973097,
        1.2130378 ,  0.41677767, -0.37284306, -0.3784905 ,  0.23228931,
        0.27963373, -0.11457254, -0.67198235, -1.2719319 , -0.06665768,
       -0.1145003 ,  0.41595927,  0.4579617 ,  0.50138754, -0.12376873,
        0.3499459 ,  0.31178138,  0.05197097, -0.8284335 , -0.15203242,
        0.02410396, -0.00348074,  0.25616574,  0.2598439 , -0.15229937,
        0.07414231, -0.7746767 , -0.8986872 , -0.10460604, -0.525213  ,
       -0.5794111 , -0.34248823, -0.36557487, -0.31474927,  0.4966923 ,
        0.1148603 , -0.48839307, -0.5833302 , -0.54751116,  0.1944994 ,
        0.31648985,  0.64005345,  0.6981818 , -0.5176394 , -0.48766503,
       -0.6015009 ,  0.45172703,  0.35486808, -0.29652384, -0.6683998 ,
        0.5481966 , -0.22577198,  0.5079388 , -0.02790587, -0.05

In [67]:
# toy_story 벡터
toy_story_vector

array([-0.02599273, -0.01448067,  0.00206561,  0.01485701, -0.00671117,
        0.0078555 ,  0.01303897,  0.04105201,  0.01287863,  0.04405019,
        0.03125044,  0.03102317,  0.03272775, -0.00715868,  0.02101746,
        0.03433464,  0.02658986,  0.00355085, -0.02518565,  0.01517774,
        0.01847286, -0.00758658,  0.0110523 , -0.02681017,  0.02056097,
        0.0103089 ,  0.02401089, -0.01786372,  0.02484703,  0.01484738,
        0.00764076,  0.01340989, -0.0033003 , -0.02722387,  0.02493652,
       -0.00461932,  0.00673437,  0.01241204, -0.00930156,  0.00101833,
        0.01218679,  0.0245075 ,  0.00539652,  0.01144614, -0.02352067,
        0.010469  , -0.02548653, -0.01731221,  0.02644469,  0.03065122,
        0.02226518,  0.01457084,  0.01875001, -0.01737853,  0.00068611,
       -0.0073206 , -0.00734477,  0.02498588, -0.00614583,  0.0162709 ,
       -0.02145315,  0.02816755,  0.00693616, -0.00254425,  0.00721628,
        0.04608276,  0.02851286,  0.01654456, -0.01022516, -0.01

In [83]:
# hojin 벡터와 toy_story 벡터 내적을 통한 선호도 파악 진행
np.dot(hojin_vector, toy_story_vector)

0.5667861

무작위로 선정한 american beauty (1999) 영화와 벡터 내적을 통한 선호도 파악 진행

In [81]:
american = movie_to_idx['american beauty (1999)']
american_vector = als_model.item_factors[american]
np.dot(hojin_vector, american_vector)

0.16188025

- hojin 벡터와 toy_story 벡터를 내적해 본 결과, 내가 선호하는 영화에선 0.5667861의 내적값을 보이고,  무작위로 선정한 american 벡터를 내적해 본 결과, 0.16188025의 내적값을 보임을 통해, 벡터의 내적값이 클 수록 hojin 이 선호할 만한 영화 임을 추측해볼 수 있습니다.  

## 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.
---

### 7-1) forrest gump (1994) 와 비슷한 영화 추천 받기

In [102]:
# 좋아하는 영화와 비슷한 영화의 인덱스 얻기
favorite_movie = 'forrest gump (1994)'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=10)
similar_movie

[(160, 1.0000001),
 (110, 0.60652167),
 (336, 0.5095103),
 (154, 0.46675903),
 (666, 0.46410143),
 (1151, 0.45613104),
 (508, 0.453468),
 (626, 0.45124596),
 (613, 0.44748613),
 (20, 0.44215488)]

In [103]:
# movie_to_idx 를 뒤집어, index로부터 movie 이름을 얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['forrest gump (1994)',
 'groundhog day (1993)',
 'pretty woman (1990)',
 'as good as it gets (1997)',
 'sleepless in seattle (1993)',
 'notting hill (1999)',
 'ghost (1990)',
 "you've got mail (1998)",
 'four weddings and a funeral (1994)',
 'pleasantville (1998)']

In [104]:
# 위 과정을 함수로 만들기
def get_similar_movie(movie_name: str):
    movie_id2 = movie_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id2)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [106]:
df = pd.DataFrame(get_similar_movie('shawshank redemption, the (1994)'))
df.columns = ['title']
df

,title
0,"shawshank redemption, the (1994)"
1,"silence of the lambs, the (1991)"
2,pulp fiction (1994)
3,schindler's list (1993)
4,good will hunting (1997)
5,fargo (1996)
6,goodfellas (1990)
7,"usual suspects, the (1995)"
8,dead man walking (1995)
9,saving private ryan (1998)


- 'hojin user' 가 선호하는 영화 리스트 중 'shawshank redemption, the (1994)'와 비슷한 영화를 찾아본 결과 비슷한 장르의 영화가 나왔음을 확인 할 수 있었습니다.

## 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.
---

### 8-1) 내가 가장 좋아할 만한 영화 추천 받기

In [107]:
user = user_to_idx['hojin']

# recommend에서는 user*item CSR Matrix를 받습니다.
# filter_already_liked_items: 유저가 이미 평가한 아이템(아티스트)는 제외

movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(121, 0.5272483),
 (110, 0.45658916),
 (141, 0.4275483),
 (50, 0.42174715),
 (48, 0.41468155),
 (23, 0.41079432),
 (38, 0.39764827),
 (107, 0.38569254),
 (87, 0.36689222),
 (175, 0.3519129),
 (51, 0.3365376),
 (62, 0.30717495),
 (200, 0.28609446),
 (322, 0.285015),
 (22, 0.27338767),
 (248, 0.27177024),
 (224, 0.2495744),
 (222, 0.23679584),
 (233, 0.2346713),
 (4, 0.22879882)]

In [109]:
df2 = pd.DataFrame([idx_to_movie[i[0]] for i in movie_recommended])
df2.columns = ['title']
df2

,title
0,"silence of the lambs, the (1991)"
1,groundhog day (1993)
2,"fugitive, the (1993)"
3,toy story 2 (1999)
4,saving private ryan (1998)
5,schindler's list (1993)
6,"sixth sense, the (1999)"
7,jurassic park (1993)
8,braveheart (1995)
9,men in black (1997)


- 개인적으로 양들의 침묵이나, 토이스토리 2, 쉰들러 리스트 와 같은 영화를 좋아하는데, 추천받은 영화 중에 제가 선호하는 영화가 있음을 확인할 수 있었습니다.

In [111]:
#  explain 메소드를 사용하면 기록을 남긴 데이터 중 이 추천에 기여한 정도를 확인
terminator_2 = movie_to_idx['terminator 2: judgment day (1991)']
explain = als_model.explain(user, csr_data, itemid=terminator_2)

In [112]:
df3 = pd.DataFrame([(idx_to_movie[i[0]], i[1]) for i in explain[1]])
df3.columns = ['title', 'Contribution']
df3

,title,Contribution
0,terminator 2: judgment day (1991),0.367900
1,"matrix, the (1999)",0.212908
2,"shawshank redemption, the (1994)",0.010052
3,toy story (1995),0.008298
4,forrest gump (1994),-0.004308


In [113]:
matrix = movie_to_idx['matrix, the (1999)']
explain = als_model.explain(user, csr_data, itemid = matrix)

In [114]:
df4 = pd.DataFrame([(idx_to_movie[i[0]], i[1]) for i in explain[1]])
df4.columns = ['title', 'Contribution']
df4

,title,Contribution
0,"matrix, the (1999)",0.297753
1,terminator 2: judgment day (1991),0.212908
2,toy story (1995),0.044122
3,"shawshank redemption, the (1994)",0.025563
4,forrest gump (1994),0.001476


- hojin 선호 영화 리스트 중, 터미네이터 2 와 매트릭스 로 끼여도를 확인해본 결과 터미네이터가 0.367900 으로 높은 기여도를 보였음을 알 수 있습니다.

## 9) 루브릭 & 회고
---

### 루브릭

1. CSR matrix가 정상적으로 만들어졌다.

- 사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었다.

2. MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다.

- 사용자와 아이템 벡터 내적수치가 의미있게 형성되었다.

3. 비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다.

- MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도가 의미있게 측정되었다.

### 회고

- 개인적으로 넷플릭스나 유튜브 알고리즘이 어떤 방식이길래 내가 좋아하는 영화나 영상을 상단에 리스트업 하는 지 개인적인 호기심을 가지고 있었습니다.

- 하지만, 그 알고리즘에 대해서 개인적으로 알아보고 싶다는 호기심을 직접 코드를 입력을 해보며 호기심을 풀 수 있었던 프로젝트였다고 생각합니다.

- 개인적인 어려움이 있었다면, 코드의 줄이 길어지면서 위에서 사용한 단어를 재사용할 때 오타로 인해 오류가 자주 발생하여 어려움을 겪었던 거 같습니다.

- 또한, user의 관심도를 측정할 수 있는 시청횟수에 따른 영화 분포를 시각화하고 싶었으나, 시간적인 제약으로 인해 실행으로 옮기지 못한 것에 아쉬움을 느낍니다.

- 항상 하는 말이지만... 시간이 넉넉하게 있었다면 다양한 시도를 할 수 있었을텐데... 라는 아쉬움을 가지며, 다음 프로젝트에선 더 좋은 결과물을 만들기 위해 노력해 보도록 하겠습니다.